<br>
<img src="images/recgov_booking.png" width=80% style="display: inline-block">
<br>
image source: http://www.recreation.gov
<br>
<img src="images/recgov_tou.png" width=80% style="display: inline-block">
<br>
image source: http://www.recreation.gov/marketing.do?goto=/acm/shareData.htm

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime
from datetime import timedelta, date
import sys
import requests

In [4]:
def check_recreationgov(facility_url, start_date, stay_length) :

    driver = webdriver.Firefox()
    
    
    
    #browser = mechanize.Browser()
    #user_agent_str = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
    #browser.addheaders = [('User-agent', user_agent_str)]	
    end_date = "{:%m/%d/%Y}".format(datetime.datetime.strptime(start_date, "%m/%d/%Y") + timedelta(1))

    try:
        driver.get(facility_url)
        #browser.open(facility_url)
    except Exception as ex:
        print("Unable to open url: " + facility_url)
        print(ex)
        return pd.DataFrame()
    
    form = browser.find_element_by_name('unifSearchForm')
    arrival = form.find_element_by_name('arrivalDate')
    departure = form.find_element_by_name('departureDate')
    arrival.send_keys(start_date)
    departure.send_keys(end_date)
    form.submit()
    #browser.select_form('unifSearchForm')
    #browser.form['arrivalDate'] = start_date
    #browser.form['departureDate'] = end_date
    #browser.submit()

    #date_resp = browser.response()
    #availability_html =  date_resp.read()

    #soup = BeautifulSoup(availability_html, 'lxml')
    soup = BeautifulSoup(browser.page_source, 'lxml')

    divs = soup.findAll('div', attrs={"class" : 'matchSummary'})
    query_result = divs[0].text
    results = pd.DataFrame()

    for filter_name in soup('div', {"class": re.compile("searchTypeFilter.*")}):
        parts = filter_name.text.split('(')
        entry_name = parts[0]

        name = entry_name.lower().strip().replace(' ', '_')
        name = name.replace("-","")

        # some names have spaces around the '-', some dont
        # i.e. standard - full vs standard-full
        # remove __ to avoid accidentally concatenating standardfull if there arent spaces
        # around the -
        name = name.replace("__", "_")
        results[name] = [parts[1].split(')')[0]]

    results = results.drop('all', axis=1)
    results['last_updated'] = ["{:%m/%d/%Y}".format(datetime.date.today())]
    results['start_date'] = start_date
    results['end_date'] = end_date

    availability_info = query_result.split(' ')
    sites_available = availability_info[0]
    total_sites = availability_info[5]

    results['sites_available'] = [sites_available]
    results['total_sites'] = [total_sites]
    return results

In [5]:
test = check_recreationgov('Slide Creek', '05/25/2016', 'http://www.recreation.gov/camping/Slide-Creek/r/campgroundDetails.do?contractCode=NRSO&parkId=72370')

WebDriverException: Message: The browser appears to have exited before we could connect. If you specified a log_file in the FirefoxBinary constructor, check it for details.
